In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, LayerNormalization, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Layer
from keras import backend as K
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations
from keras.constraints import Constraint
import pickle
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist

In [2]:
def add_salt_and_pepper_noise(images, amount):
    noisy_images = images.copy()
    num_images = images.shape[0]
    num_pixels = images.shape[1] * images.shape[2]
    num_salt = np.ceil(amount * num_pixels)
    for i in range(num_images):
        # Add salt noise
        salt_indices = np.random.choice(num_pixels, size=int(num_salt), replace=False)
        noisy_images[i].flat[salt_indices] = 1
        
        # Add pepper noise
        pepper_indices = np.random.choice(num_pixels, size=int(num_salt), replace=False)
        noisy_images[i].flat[pepper_indices] = 0
    return noisy_images

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist, cifar10

tf.random.set_seed(42)

training_acc = [0]*10
test_acc = [0]*10
test_loss = [0]*10
noise = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in range(10):  
    # load mnist dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # compute the number of labels
    num_labels = len(np.unique(y_train))

    # convert to one-hot vector
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    # resize and normalize
    image_size = X_train.shape[1]
    X_train = np.reshape(X_train,[-1, image_size, image_size])
    X_test = np.reshape(X_test,[-1, image_size, image_size])
    X_train = X_train.astype('float32') / 255
    X_test = X_test.astype('float32') / 255
    
    X_train = add_salt_and_pepper_noise(X_train, noise[i])

    input = Input(shape=(X_train.shape[1],X_train.shape[2]))
    x = SimpleRNN(512, activation='relu', return_sequences=True)(input)
    x = SimpleRNN(512, activation='relu', return_sequences=False)(x)
    x = Dense(num_labels, activation='softmax')(x)
    model = Model(input, x)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_split=0.25, verbose=2)

    training_acc[i] = history.history['val_accuracy'][-1]
    print(training_acc)
    
    loss, acc = model.evaluate(X_test, y_test, batch_size=128)
    test_acc[i] = acc
    print(test_acc)

    name = 'rnn_512_'
    name = name + str(noise[i]) + '.h5'
    model.save(name)

model.summary()

Epoch 1/100
352/352 - 141s - loss: 0.3803 - accuracy: 0.8777 - val_loss: 0.1699 - val_accuracy: 0.9517 - 141s/epoch - 399ms/step
Epoch 2/100
352/352 - 112s - loss: 0.1323 - accuracy: 0.9632 - val_loss: 0.1339 - val_accuracy: 0.9609 - 112s/epoch - 319ms/step
Epoch 3/100
352/352 - 112s - loss: 0.1110 - accuracy: 0.9695 - val_loss: 0.1120 - val_accuracy: 0.9681 - 112s/epoch - 319ms/step
Epoch 4/100
352/352 - 115s - loss: 0.0956 - accuracy: 0.9742 - val_loss: 0.0978 - val_accuracy: 0.9751 - 115s/epoch - 326ms/step
Epoch 5/100
352/352 - 113s - loss: 0.0766 - accuracy: 0.9789 - val_loss: 0.0981 - val_accuracy: 0.9765 - 113s/epoch - 321ms/step
Epoch 6/100
352/352 - 113s - loss: 0.0701 - accuracy: 0.9810 - val_loss: 0.0844 - val_accuracy: 0.9751 - 113s/epoch - 321ms/step
Epoch 7/100
352/352 - 115s - loss: 0.0738 - accuracy: 0.9797 - val_loss: 0.0886 - val_accuracy: 0.9755 - 115s/epoch - 326ms/step
Epoch 8/100
352/352 - 115s - loss: 0.0629 - accuracy: 0.9828 - val_loss: 0.1266 - val_accuracy: 0

Epoch 65/100
352/352 - 106s - loss: 0.0198 - accuracy: 0.9944 - val_loss: 0.0939 - val_accuracy: 0.9826 - 106s/epoch - 302ms/step
Epoch 66/100
352/352 - 103s - loss: 0.0172 - accuracy: 0.9956 - val_loss: 0.0725 - val_accuracy: 0.9857 - 103s/epoch - 293ms/step
Epoch 67/100
352/352 - 102s - loss: 0.0122 - accuracy: 0.9967 - val_loss: 0.0957 - val_accuracy: 0.9821 - 102s/epoch - 289ms/step
Epoch 68/100
352/352 - 103s - loss: 0.0201 - accuracy: 0.9951 - val_loss: 0.0766 - val_accuracy: 0.9840 - 103s/epoch - 293ms/step
Epoch 69/100
352/352 - 105s - loss: 0.0164 - accuracy: 0.9954 - val_loss: 0.0860 - val_accuracy: 0.9825 - 105s/epoch - 299ms/step
Epoch 70/100
352/352 - 105s - loss: 0.0337 - accuracy: 0.9912 - val_loss: 0.0774 - val_accuracy: 0.9823 - 105s/epoch - 298ms/step
Epoch 71/100
352/352 - 96s - loss: 0.0280 - accuracy: 0.9922 - val_loss: 0.0734 - val_accuracy: 0.9833 - 96s/epoch - 274ms/step
Epoch 72/100
352/352 - 90s - loss: 0.0153 - accuracy: 0.9959 - val_loss: 0.0813 - val_accura

C:\Users\zihaow19\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/100
352/352 - 158s - loss: 0.6082 - accuracy: 0.8010 - val_loss: 0.3750 - val_accuracy: 0.8838 - 158s/epoch - 450ms/step
Epoch 2/100
352/352 - 106s - loss: 0.2257 - accuracy: 0.9312 - val_loss: 0.1941 - val_accuracy: 0.9420 - 106s/epoch - 300ms/step
Epoch 3/100
352/352 - 89s - loss: 0.1651 - accuracy: 0.9512 - val_loss: 0.1798 - val_accuracy: 0.9473 - 89s/epoch - 254ms/step
Epoch 4/100
352/352 - 91s - loss: 0.1437 - accuracy: 0.9579 - val_loss: 0.2154 - val_accuracy: 0.9391 - 91s/epoch - 260ms/step
Epoch 5/100
352/352 - 95s - loss: 0.1185 - accuracy: 0.9651 - val_loss: 0.1669 - val_accuracy: 0.9527 - 95s/epoch - 271ms/step
Epoch 6/100
352/352 - 97s - loss: 0.1027 - accuracy: 0.9698 - val_loss: 0.1540 - val_accuracy: 0.9549 - 97s/epoch - 275ms/step
Epoch 7/100
352/352 - 77s - loss: 0.0941 - accuracy: 0.9715 - val_loss: 0.2068 - val_accuracy: 0.9423 - 77s/epoch - 219ms/step
Epoch 8/100
352/352 - 96s - loss: 0.0910 - accuracy: 0.9733 - val_loss: 0.1591 - val_accuracy: 0.9569 - 96s

Epoch 65/100
352/352 - 97s - loss: 0.0228 - accuracy: 0.9934 - val_loss: 0.2130 - val_accuracy: 0.9619 - 97s/epoch - 276ms/step
Epoch 66/100
352/352 - 97s - loss: 0.0404 - accuracy: 0.9893 - val_loss: 0.1938 - val_accuracy: 0.9585 - 97s/epoch - 277ms/step
Epoch 67/100
352/352 - 98s - loss: 0.0316 - accuracy: 0.9912 - val_loss: 0.2026 - val_accuracy: 0.9601 - 98s/epoch - 277ms/step
Epoch 68/100
352/352 - 96s - loss: 0.0265 - accuracy: 0.9929 - val_loss: 0.1886 - val_accuracy: 0.9629 - 96s/epoch - 273ms/step
Epoch 69/100
352/352 - 97s - loss: 0.0294 - accuracy: 0.9914 - val_loss: 0.2053 - val_accuracy: 0.9602 - 97s/epoch - 276ms/step
Epoch 70/100
352/352 - 98s - loss: 0.0191 - accuracy: 0.9946 - val_loss: 0.2118 - val_accuracy: 0.9612 - 98s/epoch - 279ms/step
Epoch 71/100
352/352 - 96s - loss: 0.0295 - accuracy: 0.9920 - val_loss: 0.2096 - val_accuracy: 0.9589 - 96s/epoch - 274ms/step
Epoch 72/100
352/352 - 98s - loss: 0.0453 - accuracy: 0.9873 - val_loss: 0.1902 - val_accuracy: 0.9547 -

...

Epoch 30/100
352/352 - 59s - loss: 0.4428 - accuracy: 0.8501 - val_loss: 5.1196 - val_accuracy: 0.1373 - 59s/epoch - 167ms/step
Epoch 31/100
352/352 - 59s - loss: 0.4262 - accuracy: 0.8561 - val_loss: 5.0593 - val_accuracy: 0.1351 - 59s/epoch - 166ms/step
Epoch 32/100
352/352 - 60s - loss: 0.4202 - accuracy: 0.8579 - val_loss: 5.3463 - val_accuracy: 0.1348 - 60s/epoch - 169ms/step
Epoch 33/100
352/352 - 59s - loss: 0.4206 - accuracy: 0.8578 - val_loss: 5.2141 - val_accuracy: 0.1325 - 59s/epoch - 168ms/step
Epoch 34/100
352/352 - 58s - loss: 0.3955 - accuracy: 0.8650 - val_loss: 5.3812 - val_accuracy: 0.1357 - 58s/epoch - 166ms/step
Epoch 35/100
352/352 - 59s - loss: 0.3900 - accuracy: 0.8682 - val_loss: 5.2104 - val_accuracy: 0.1409 - 59s/epoch - 167ms/step
Epoch 36/100
352/352 - 60s - loss: 0.3965 - accuracy: 0.8664 - val_loss: 5.2697 - val_accuracy: 0.1363 - 60s/epoch - 169ms/step
Epoch 37/100
352/352 - 58s - loss: 0.4373 - accuracy: 0.8513 - val_loss: 5.2440 - val_accuracy: 0.1355 -

352/352 - 70s - loss: 0.3197 - accuracy: 0.8936 - val_loss: 5.8827 - val_accuracy: 0.1287 - 70s/epoch - 199ms/step
Epoch 95/100
352/352 - 73s - loss: 0.3366 - accuracy: 0.8870 - val_loss: 5.9652 - val_accuracy: 0.1278 - 73s/epoch - 209ms/step
Epoch 96/100
352/352 - 74s - loss: 0.3437 - accuracy: 0.8854 - val_loss: 5.9420 - val_accuracy: 0.1354 - 74s/epoch - 211ms/step
Epoch 97/100
352/352 - 72s - loss: 0.3414 - accuracy: 0.8866 - val_loss: 5.9537 - val_accuracy: 0.1321 - 72s/epoch - 204ms/step
Epoch 98/100
352/352 - 72s - loss: 0.3509 - accuracy: 0.8839 - val_loss: 5.8978 - val_accuracy: 0.1347 - 72s/epoch - 205ms/step
Epoch 99/100
352/352 - 78s - loss: 0.3375 - accuracy: 0.8873 - val_loss: 5.9409 - val_accuracy: 0.1315 - 78s/epoch - 223ms/step
Epoch 100/100
352/352 - 80s - loss: 0.3406 - accuracy: 0.8877 - val_loss: 5.7929 - val_accuracy: 0.1335 - 80s/epoch - 227ms/step
[0.9773333072662354, 0.9620000123977661, 0.9118666648864746, 0.8255333304405212, 0.6524666547775269, 0.4235333204269

352/352 - 141s - loss: 0.3459 - accuracy: 0.8830 - val_loss: 5.9496 - val_accuracy: 0.1037 - 141s/epoch - 399ms/step
Epoch 55/100
352/352 - 156s - loss: 0.3449 - accuracy: 0.8851 - val_loss: 6.0467 - val_accuracy: 0.1025 - 156s/epoch - 443ms/step
Epoch 56/100
352/352 - 154s - loss: 0.3695 - accuracy: 0.8734 - val_loss: 6.0002 - val_accuracy: 0.1029 - 154s/epoch - 436ms/step
Epoch 57/100
352/352 - 155s - loss: 0.3471 - accuracy: 0.8832 - val_loss: 5.9909 - val_accuracy: 0.1012 - 155s/epoch - 439ms/step
Epoch 58/100
352/352 - 153s - loss: 0.3332 - accuracy: 0.8891 - val_loss: 6.0078 - val_accuracy: 0.0985 - 153s/epoch - 435ms/step
Epoch 59/100
352/352 - 150s - loss: 0.3145 - accuracy: 0.8932 - val_loss: 6.0984 - val_accuracy: 0.1007 - 150s/epoch - 426ms/step
Epoch 60/100
352/352 - 148s - loss: 0.3327 - accuracy: 0.8871 - val_loss: 6.0578 - val_accuracy: 0.0972 - 148s/epoch - 422ms/step
Epoch 61/100
352/352 - 150s - loss: 0.3502 - accuracy: 0.8813 - val_loss: 6.0873 - val_accuracy: 0.1009

Epoch 14/100
352/352 - 109s - loss: 0.9035 - accuracy: 0.6967 - val_loss: 3.8324 - val_accuracy: 0.1010 - 109s/epoch - 311ms/step
Epoch 15/100
352/352 - 111s - loss: 0.8300 - accuracy: 0.7214 - val_loss: 3.9832 - val_accuracy: 0.1017 - 111s/epoch - 314ms/step
Epoch 16/100
352/352 - 110s - loss: 0.7862 - accuracy: 0.7342 - val_loss: 4.0681 - val_accuracy: 0.1019 - 110s/epoch - 314ms/step
Epoch 17/100
352/352 - 112s - loss: 0.7352 - accuracy: 0.7532 - val_loss: 4.1957 - val_accuracy: 0.0993 - 112s/epoch - 318ms/step
Epoch 18/100
352/352 - 110s - loss: 0.6870 - accuracy: 0.7714 - val_loss: 4.4181 - val_accuracy: 0.1002 - 110s/epoch - 312ms/step
Epoch 19/100
352/352 - 111s - loss: 0.6731 - accuracy: 0.7755 - val_loss: 4.4240 - val_accuracy: 0.1021 - 111s/epoch - 314ms/step
Epoch 20/100
352/352 - 112s - loss: 0.6505 - accuracy: 0.7810 - val_loss: 4.5210 - val_accuracy: 0.1025 - 112s/epoch - 319ms/step
Epoch 21/100
352/352 - 110s - loss: 0.6266 - accuracy: 0.7885 - val_loss: 4.6409 - val_acc

352/352 - 221s - loss: 0.3603 - accuracy: 0.8784 - val_loss: 5.9686 - val_accuracy: 0.0977 - 221s/epoch - 628ms/step
Epoch 78/100
352/352 - 195s - loss: 0.4178 - accuracy: 0.8598 - val_loss: 5.8889 - val_accuracy: 0.1009 - 195s/epoch - 554ms/step
Epoch 79/100
352/352 - 189s - loss: 0.4145 - accuracy: 0.8611 - val_loss: 5.8660 - val_accuracy: 0.0970 - 189s/epoch - 538ms/step
Epoch 80/100
352/352 - 192s - loss: 0.3948 - accuracy: 0.8672 - val_loss: 5.8641 - val_accuracy: 0.1015 - 192s/epoch - 545ms/step
Epoch 81/100
352/352 - 214s - loss: 0.3770 - accuracy: 0.8749 - val_loss: 5.9210 - val_accuracy: 0.1001 - 214s/epoch - 608ms/step
Epoch 82/100
352/352 - 219s - loss: 0.3747 - accuracy: 0.8745 - val_loss: 5.9352 - val_accuracy: 0.0977 - 219s/epoch - 623ms/step
Epoch 83/100
352/352 - 224s - loss: 0.3866 - accuracy: 0.8705 - val_loss: 5.9746 - val_accuracy: 0.1004 - 224s/epoch - 636ms/step
Epoch 84/100
352/352 - 221s - loss: 0.4115 - accuracy: 0.8607 - val_loss: 5.9821 - val_accuracy: 0.0999

In [4]:
from tensorflow.python.framework.convert_to_constants import  convert_variables_to_constants_v2_as_graph

def get_flops(model):
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func, lower_control_flow=False)
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return flops.total_float_ops

print("The FLOPs is:{}".format(get_flops(model)) ,flush=True )

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
The FLOPs is:1613884
